In [17]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import requests
import pandas as pd
import json
pd.set_option('display.max_rows',None)

## API 参数研究

In [6]:
import requests
import pandas as pd
import json
pd.set_option('display.max_columns',None)


# PHIA Netherlands
# DSM Netherlands

# all primary dr in netherlands & america

country = 'uk'
columns = '["id","name","description","type","close","currency","change","Value.Traded","market_cap_basic","fundamental_currency_code","sector","industry","market","is_primary","exchange","country"]'
payload = '{"columns":'+ columns +',"filter":[{"left":"name","operation":"nempty"},{"left": "type", "operation": "nequal", "right": "fund"}],"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,15000],"sort":{"sortBy":"market_cap_basic","sortOrder":"desc"}}'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}


url = 'https://scanner.tradingview.com/' + country + '/scan'
r = requests.post(url, headers = headers, data = payload).text
df = pd.DataFrame(json.loads(r)['data'])['d']
df = pd.DataFrame(df.tolist(), index = df.index, columns = json.loads(columns))
df['turnover_rate'] = (df['Value.Traded'] / df['market_cap_basic']).round(4) 
df = df[( ((df['is_primary'] == True) & (df['type'] == 'stock')) | \
          ((df['is_primary'] == True) & (df['type'] == 'dr') & (df['market'].isin(['netherlands','america']))) | \
          ((df['name'].isin(['PHIA','DSM'])) & (df['market'].isin(['netherlands']))) ) & \
        (~df['exchange'].isin(['OTC']))]
        
df

KeyError: 'd'

In [7]:
json.loads(r)

{'totalCount': 0, 'error': 'Unknown field "id"', 'data': None}

## 完整代码

In [80]:
import requests
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
'''
fvey = america,australia,canada,newzealand,uk
EUR = belgium,france,germany,italy,luxembourg,netherlands,portugal,spain,switzerland*
北欧 = iceland,denmark,finland,norway,sweden

拉美 = argentina,brazil,chile,colombia,mexico,peru,venezuela
中东 = bahrain,egypt,israel,kuwait,qatar,turkey,uae,ksa(Saudi Arabia)
非洲 = kenya,morocco,nigeria,tunisia,rsa(South Africa)
东盟 = indonesia,malaysia,philippines,singapore,thailand,vietnam
南亚 = india,pakistan,srilanka

第一岛链 = japan,korea,taiwan

china,hongkong

russia,lithuania,latvia,estonia,serbia,hungary,romania,poland,slovakia,cyprus,greece

'''

fvey = ['america','australia','canada','newzealand','uk']
eur = ['belgium','france','germany','italy','luxembourg','netherlands','portugal','spain','switzerland','cyprus','greece']
n_eur = ['iceland','denmark','finland','norway','sweden']
china = ['china','hongkong']
east_asia = ['japan','korea','taiwan']
latin_america = ['argentina','brazil','chile','colombia','mexico','peru','venezuela']
middle_east = ['bahrain','egypt','israel','kuwait','qatar','turkey','uae','ksa']
africa = ['kenya','morocco','nigeria','tunisia','rsa']
asan = ['indonesia','malaysia','philippines','singapore','thailand','vietnam']
indian = ['india','pakistan','srilanka']
east_eur = ['russia','lithuania','latvia','estonia','serbia','hungary','romania','poland','slovakia']


country_list = fvey + eur + n_eur + china + east_asia + latin_america + middle_east + africa + asan + indian + east_eur

def get_USD_forex_table():
    url = 'https://scanner.tradingview.com/forex/scan'
    payload = '{"columns":["currency_logoid","base_currency_logoid","name","description","update_mode","type","typespecs","close","pricescale","minmov","fractional","minmove2","currency","change","change_abs","bid","ask","high","low","Recommend.All"],"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,2000],"sort":{"sortBy":"name","sortOrder":"asc","nullsFirst":false},"preset":"forex_rates_all"}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

    r = requests.post(url, headers = headers, data = payload).text
    forex_df = pd.DataFrame(json.loads(r)['data'])['d']
    forex_df = pd.DataFrame(forex_df.tolist(), index = forex_df.index)
    USD_forex_table = forex_df[(forex_df[12] == 'USD')][[2,3,7]]
    USD_forex_table['currency_id'] =  USD_forex_table[2].str.replace('USD','')
    USD_forex_table['name'] =  USD_forex_table[3].str.replace(' / U.S. DOLLAR','')
    USD_forex_table['forex_rate'] =  USD_forex_table[7]
    USD_forex_table = USD_forex_table.reset_index(inplace = False)[['currency_id','forex_rate']]
#     USD_forex_table = USD_forex_table.append({'currency_id':'USD','forex_rate':1}, ignore_index=True)
    USD_forex_table = pd.concat([USD_forex_table, pd.DataFrame({'currency_id':['USD'],'forex_rate':[1]})], ignore_index=True)
    return USD_forex_table

def fetch_country_data(country_list):
    
    # columns = ["name","description","logoid","update_mode","type","typespecs","close","pricescale","minmov","fractional","minmove2","currency","change","change_abs","Recommend.All","volume","Value.Traded","market_cap_basic","fundamental_currency_code","price_earnings_ttm","earnings_per_share_basic_ttm","number_of_employees_fy","sector","industry","market"]
    columns = '["name","description","type","close","currency","change","Value.Traded","market_cap_basic","fundamental_currency_code","sector","industry","market"]'
    payload = '{"columns":'+ columns +',"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,9000],"sort":{"sortBy":"name","sortOrder":"asc","nullsFirst":false},"preset":"all_stocks"}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    
    all_country_df = pd.DataFrame()
    for country in country_list:
#         print(country)
        url = 'https://scanner.tradingview.com/' + country + '/scan'
        r = requests.post(url, headers = headers, data = payload).text
        df = pd.DataFrame(json.loads(r)['data'])['d']
        df = pd.DataFrame(df.tolist(), index = df.index, columns = json.loads(columns))
        df['Value.Traded'] = df['Value.Traded']/100 if country in ['uk','kuwait','rsa','israel','turkey'] else df['Value.Traded']  # 修正英国市场成交量
        df['turnover_rate'] = (df['Value.Traded'] / df['market_cap_basic']).round(4) 
        df = df.merge(USD_forex_table,left_on='fundamental_currency_code', right_on = 'currency_id')
        df = df[~df['change'].isnull()]
        df['Value.Traded'] = (df['Value.Traded']* df['forex_rate']/1000000).fillna(0) 
        df = df[df['Value.Traded'] > 0]
        df['market_cap_USD'] = df['market_cap_basic'] * df['forex_rate']
        df['market_cap_USD'] = (df['market_cap_USD']/100000000).fillna(0) 
        df = df[df['market_cap_USD'] > 0]
        all_country_df = pd.concat([all_country_df, df], ignore_index=True)
    return all_country_df

# modified payload params and filters
# def fetch_country_data(country_list):
    
#     # columns = ["name","description","logoid","update_mode","type","typespecs","close","pricescale","minmov","fractional","minmove2","currency","change","change_abs","Recommend.All","volume","Value.Traded","market_cap_basic","fundamental_currency_code","price_earnings_ttm","earnings_per_share_basic_ttm","number_of_employees_fy","sector","industry","market"]
#     columns = '["name","description","type","close","currency","change","Value.Traded","market_cap_basic","fundamental_currency_code","sector","industry","market","is_primary","exchange","country"]'
#     payload = '{"columns":'+ columns +',"filter":[{"left":"name","operation":"nempty"},{"left":"typespecs","operation":"has_none_of","right":"preferred"},{"left": "type", "operation": "nequal", "right": "fund"}],"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,15000],"sort":{"sortBy":"market_cap_basic","sortOrder":"desc"}}'
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    
#     all_country_df = pd.DataFrame()
#     for country in country_list:
# #         print(country)
#         url = 'https://scanner.tradingview.com/' + country + '/scan'
#         r = requests.post(url, headers = headers, data = payload).text
#         df = pd.DataFrame(json.loads(r)['data'])['d']
#         df = pd.DataFrame(df.tolist(), index = df.index, columns = json.loads(columns))
#         df['Value.Traded'] = df['Value.Traded']/100 if country in ['uk'] else df['Value.Traded']  # 修正英国市场成交量        
#         df['turnover_rate'] = (df['Value.Traded'] / df['market_cap_basic']).round(4) 
#         df = df[( ((df['is_primary'] == True) & (df['type'] == 'stock')) | \
#                   ((df['is_primary'] == True) & (df['type'] == 'dr') & (df['market'].isin(['netherlands','america']))) | \
#                   ((df['name'].isin(['PHIA','DSM'])) & (df['market'].isin(['netherlands']))) ) & \
#                 (~df['exchange'].isin(['OTC']))]
        
#         df = df.merge(USD_forex_table,left_on='fundamental_currency_code', right_on = 'currency_id')
#         df = df[~df['change'].isnull()]
#         df['Value.Traded'] = (df['Value.Traded']* df['forex_rate']/1000000).fillna(0) 
#         df = df[df['Value.Traded'] > 0]
#         df['market_cap_USD'] = df['market_cap_basic'] * df['forex_rate']
#         df['market_cap_USD'] = (df['market_cap_USD']/100000000).fillna(0) 
#         df = df[df['market_cap_USD'] > 0]
        
#         all_country_df = pd.concat([all_country_df, df], ignore_index=True)
#     return all_country_df

USD_forex_table = get_USD_forex_table()
df = fetch_country_data(country_list)

# fig = px.treemap(df, 
#                  path=['market','sector'],  # 指定层次结构，每一个层次都应该是category型的变量
#                  values='market_cap_USD', # 需要聚合的列名
#                  color='change', 
#                  custom_data=['change','name','market_cap_USD','close'],
#                  range_color = [-8, 8], # 色彩范围最大最小值
# #                  hover_data= {'涨跌幅':':.2',
# #                              '总市值':':.1f'}, # 鼠标悬浮显示数据的格式
#                  color_continuous_scale=["seagreen",'lightgrey', "indianred"],
#                  height = 900,
#                 #  width = 1600,
#                  color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
#                 )
# fig.update_coloraxes(showscale=False)
# fig.update_traces(textposition='middle center', 
#                   textinfo='label', 
#                   textfont = dict(color='white'),
#                   texttemplate= "%{label}<br>%{customdata[0]:.2f}%<br>",
#                   hovertemplate= "%{label}<br>%{customdata[3]:.2f}  (%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿"                  
# #                   hovertemplate= "%{customdata[1]}<br>%{label}<br>(%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿"
#                  ) 
# fig.show()
df.to_csv('world_stock.csv',index = False, encoding = 'utf-8')

In [82]:
import plotly.express as px
import pandas as pd

df = pd.read_csv('world_stock.csv', encoding = 'utf-8')



def check_plate(market):
    if market in ['america']:
        market = 'USA'
    elif market in (fvey + eur + n_eur):
        market = 'Western'
    elif market in east_eur:
        market = 'East Europe'
    elif market in china:
        market = 'China'
    elif market in east_asia:
        market = 'East Asia'
    elif market in latin_america :
        market = 'Latin'
    elif market in middle_east + africa:
        market = 'Middle East + Africa'
    elif market in indian:
        market = 'South Asia'
    elif market in asan:
        market = 'ASEAN'
    else:
        market = 'Rest'
    return market

fvey = ['australia','canada','newzealand','uk']
eur = ['belgium','france','germany','italy','luxembourg','netherlands','portugal','spain','switzerland','cyprus','greece']
n_eur = ['iceland','denmark','finland','norway','sweden']
china = ['china','hongkong']
east_asia = ['japan','korea','taiwan']
latin_america = ['argentina','brazil','chile','colombia','mexico','peru','venezuela']
middle_east = ['bahrain','egypt','israel','kuwait','qatar','turkey','uae','ksa']
africa = ['kenya','morocco','nigeria','tunisia','rsa']
asan = ['indonesia','malaysia','philippines','singapore','thailand','vietnam']
indian = ['india','pakistan','srilanka']
east_eur = ['russia','lithuania','latvia','estonia','serbia','hungary','romania','poland','slovakia']


df['plate'] = df['market'].apply(check_plate)
df['turnover_USD'] = df['market_cap_USD'] * df['turnover_rate']
df = df[df['turnover_USD'] > 0.01]

fig = px.treemap(df, 
                 path=['plate','market','sector'],  # 指定层次结构，每一个层次都应该是category型的变量
#                  path=['plate','','sector','industry',],
                 values='turnover_USD', # 需要聚合的列名
                 color='change', 
                 custom_data=['change','description','turnover_USD','close','market', 'market_cap_USD'],
                 range_color = [-8, 8], # 色彩范围最大最小值
#                  hover_data= {'涨跌幅':':.2',
#                              '总市值':':.1f'}, # 鼠标悬浮显示数据的格式
                 color_continuous_scale=["seagreen",'lightgrey', "indianred"],
                 height = 900,
                #  width = 1600,
                 color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
                )
fig.update_coloraxes(showscale=False)
fig.update_traces(textposition='middle center', 
                  textinfo='label', 
                  textfont = dict(color='white'),
                  texttemplate= "%{label}<br>%{customdata[0]:.2f}%<br>",
                  hovertemplate= "%{label}<br>%{customdata[0]:.2f}%<br>%{customdata[4]}<br>交易额=%{customdata[2]:d}亿<br>总市值=%{customdata[5]:d}亿"                  
#                   hovertemplate= "%{customdata[1]}<br>%{label}<br>(%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿"
                 ) 
fig.show()

## 分板块详情

## Global Data

In [35]:
import requests
import pandas as pd
import json

# PHIA Netherlands
# DSM Netherlands
# all primary dr in netherlands & america

def check_plate(market):
    fvey = ['america','canada']
    eur = ['uk','belgium','france','germany','italy','luxembourg','netherlands','portugal','spain','switzerland','cyprus','greece']
    n_eur = ['iceland','denmark','finland','norway','sweden']
    china = ['china','hongkong']
    east_asia = ['japan','korea','taiwan','australia','newzealand']
    latin_america = ['argentina','brazil','chile','colombia','mexico','peru','venezuela']
    middle_east = ['bahrain','egypt','israel','kuwait','qatar','turkey','uae','ksa']
    africa = ['kenya','morocco','nigeria','tunisia','rsa']
    asan = ['indonesia','malaysia','philippines','singapore','thailand','vietnam']
    indian = ['india','pakistan','srilanka']
    east_eur = ['russia','lithuania','latvia','estonia','serbia','hungary','romania','poland','slovakia']  
    
    if market in fvey:
        market = '北美'
    elif market in (eur + n_eur):
        market = '欧洲'
#     elif market in east_eur:
#         market = 'East Europe'
    elif market in china:
        market = '中国'
    elif market in east_asia:
        market = '亚太'
    elif market in latin_america :
        market = '拉丁美洲'
    elif market in middle_east + africa + east_eur:
        market = '中东/非'
    elif market in indian:
        market = '南亚'
    elif market in asan:
        market = '东盟'
    else:
        market = 'Rest'
    return market



def get_USD_forex_table():
    url = 'https://scanner.tradingview.com/forex/scan'
    payload = '{"columns":["currency_logoid","base_currency_logoid","name","description","update_mode","type","typespecs","close","pricescale","minmov","fractional","minmove2","currency","change","change_abs","bid","ask","high","low","Recommend.All"],"ignore_unknown_fields":false,"options":{"lang":"en"},"range":[0,2000],"sort":{"sortBy":"name","sortOrder":"asc","nullsFirst":false},"preset":"forex_rates_all"}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}

    r = requests.post(url, headers = headers, data = payload).text
    forex_df = pd.DataFrame(json.loads(r)['data'])['d']
    forex_df = pd.DataFrame(forex_df.tolist(), index = forex_df.index)
    USD_forex_table = forex_df[(forex_df[12] == 'USD')][[2,3,7]]
    USD_forex_table['currency_id'] =  USD_forex_table[2].str.replace('USD','').str.replace('GBP','GBX')
    USD_forex_table['name'] =  USD_forex_table[3].str.replace(' / U.S. DOLLAR','')
    USD_forex_table['forex_rate'] =  USD_forex_table[7]
    USD_forex_table = USD_forex_table.reset_index(inplace = False)[['currency_id','forex_rate']]
#     USD_forex_table = USD_forex_table.append({'currency_id':'USD','forex_rate':1}, ignore_index=True)
    USD_forex_table = pd.concat([USD_forex_table, pd.DataFrame({'currency_id':['USD'],'forex_rate':[1]})], ignore_index=True)

    
    USD_forex_table.loc[USD_forex_table['currency_id'].isin(['GBX']), 'forex_rate'] = USD_forex_table[USD_forex_table['currency_id'].isin(['GBX'])]['forex_rate']/100 
    return USD_forex_table

def fetch_global_data():
    
    country = 'global'
    columns = '["name","description","logoid","type","close","currency","change","Value.Traded","market_cap_basic","fundamental_currency_code","sector","industry","market","is_primary","exchange","country"]'
    payload = '{"columns":'+ columns +',"filter":[{"left":"typespecs","operation":"has_none_of","right":["etn","etf"]}],"ignore_unknown_fields":false,"price_conversion":{"to_currency":"usd"},"range":[0,90000],"sort":{"sortBy":"market_cap_basic","sortOrder":"desc"},"markets":["america","argentina","australia","bahrain","belgium","brazil","canada","chile","china","colombia","cyprus","denmark","egypt","estonia","finland","france","germany","greece","hongkong","hungary","iceland","india","indonesia","israel","italy","japan","kenya","kuwait","latvia","lithuania","luxembourg","malaysia","mexico","morocco","netherlands","newzealand","nigeria","norway","pakistan","peru","philippines","poland","portugal","qatar","romania","russia","ksa","serbia","singapore","slovakia","rsa","korea","spain","srilanka","sweden","switzerland","taiwan","thailand","tunisia","turkey","uae","uk","venezuela","vietnam"]}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    
    url = 'https://scanner.tradingview.com/' + country + '/scan'
    r = requests.post(url, headers = headers, data = payload).text
    df = pd.DataFrame(json.loads(r)['data'])['d']
    df = pd.DataFrame(df.tolist(), index = df.index, columns = json.loads(columns))
    
    USD_forex_table = get_USD_forex_table()
    df = df.merge(USD_forex_table,left_on='currency', right_on = 'currency_id')
    
    df['market_cap_USD'] = (df['market_cap_basic']/100000000).fillna(0)
    df = df[df['market_cap_USD'] > 0]
    
    df['Traded_USD'] = (df['Value.Traded']* df['forex_rate']/100000000).fillna(0)
    df = df[df['Value.Traded'] > 0.01]
    
    df['turnover_rate'] = (df['Traded_USD'] / df['market_cap_USD']).round(4)
    
    df['plate'] = df['market'].apply(check_plate)
    df = df[~df['change'].isnull()]
    df = df[~df['industry'].isnull()]
    df = df[~df['sector'].isnull()]
    df = df[~df['market_cap_basic'].isnull()]
    
    return df



df = fetch_global_data()


        
# df.head(100)

In [2]:
a_df = df[( ((df['is_primary'] == True) & (df['type'] == 'stock')) | \
          ((df['is_primary'] == True) & (df['type'] == 'dr') & (df['market'].isin(['netherlands','america']))) | \
          ((df['name'].isin(['PHIA','DSM'])) & (df['market'].isin(['netherlands']))) ) & \
        (~df['exchange'].isin(['OTC']))]

In [ ]:
from pygtrans import Translate

client = Translate()


def translate(text):
    try:
        result = client.translate(text)
        print(result.translatedText)
        return result.translatedText 
    except:
        print(text)
        return text
    
a_df = a_df[a_df['market'].isin(['japan'])]

a_df['公司名名称'] = a_df['description'].apply(translate) 


丰田汽车公司
基恩士公司
索尼集团公司
日本电话电话公司
三菱UFJ金融集团
迅销有限公司
三菱公司
东京电子
凯迪迪公司
信越化学工业株式会社
第一三协株式会社
东方置业
大金工业
三井物产
伊藤忠商事株式会社
软银集团公司
日立
三井住友金融集团
本田汽车公司
瑞瑞控股有限公司
任天堂有限公司
武田制药有限公司
软银公司
电装公司
中外制药公司
东京海洋控股公司
霍亚公司
日本烟草公司
SMC株式会社
七一控股有限公司
瑞穗金融集团
村田制作所
发那科公司
瑞萨电子公司
日本电产株式会社
三菱电机公司
丸红株式会社
普利司通公司
日本邮政银行
安斯泰来制药
佳能公司
松下控股公司
住友商事株式会社
爱德万测试公司
小松
东海旅客铁道
泰尔茂株式会社
富士通
日本邮政控股株式会社
尤妮佳公司
丰田工业公司
大冢控股有限公司
味之素株式会社
东日本旅客铁道公司
奥林巴斯公司
欧力士公司
NTT数据公司
朝日集团控股
京瓷公司
日本涂料控股有限公司
资生堂有限公司
卫材公司
Z控股公司
三井不动产
新日铁公司
第一生命控股有限公司
富士胶片控股公司
MS&amp;AD INS GP 控股
久保田公司
迪斯科公司
丰田通商株式会社
阿斯公司
大和房屋工业株式会社
尼克森有限公司
铃木汽车公司
永旺有限公司
三菱重工
野村综合研究所
三菱地所株式会社
万代南梦宫控股有限公司
M3公司
禧玛诺公司
英派克斯公司
森宝控股公司
TDK株式会社
西科姆公司
日产汽车公司
奥比克有限公司
希森美康公司
斯巴鲁公司
激光技术公司
东芝公司
福尔控股有限公司
日本电气公司
三井住友信托控股公司
盐野木公司
信用金中央银行非累积 PRF 100000 日元
积水屋
安川电机公司
麒麟控股有限公司
住友不动产开发公司
欧姆龙公司
野村控股株式会社
三得利饮料食品有限公司
龟甲万公司
瑞索纳控股
日东电工株式会社
关西电力公司
全日空控股公司
日本邮船株式会社
日本甲骨文公司
泛太平洋国际控股公司
西日本旅客铁道公司
协和麒麟株式会社
养乐多本社株式会社
英力士控股公司
雅马哈汽车公司
东京煤气公司
五十铃汽车
旭化成株式会社
日本酸素控股株式会社
住友电工
小野制药公司
日本航空
罗姆株式会社
SG控股有限公司
SHIMADZU CORP
日本交流集团
住友金属矿业公司
中部电力公司
东

富士屋有限公司
向量公司
肯德基控股日本有限公司
福井电脑控股有限公司
石油公司
青山贸易公司
林格小屋有限公司
TSI控股有限公司
新闻工业株式会社
冲绳电力公司
传康有限公司
坂田INX株式会社
特莱控股公司
东和株式会社
计算机工程与咨询
爱知县株式会社
第一实业公司
日精ASB机械公司
金冠有限公司
利优比有限公司
布什路公司
日本烧金工业株式会社
松屋公司（东京）
大研株式会社
长濑兄弟株式会社
医药食品国际公司
丸花故乡株式会社
野村微科技有限公司
维特控股有限公司
MTG有限公司
裕泰公司
坂东化学工业有限公司
拓泰克公司
古河公司
奥普廷公司
托克有限公司
帕克公司
住友精化化学株式会社
交易媒体网络公司
J信托有限公司
联合工具有限公司
瑞森有限公司
安旺集团公司
北斗株式会社
EM系统有限公司
东亚株式会社
普罗克雷亚控股有限公司
软件服务公司
歌舞伎座株式会社
K&amp;O能源集团有限公司
三井住友建设株式会社
DAI-DAN CO LTD
三重交通集团控股株式会社
游戏卡-JOYCO HLDGS INC
日本精机株式会社
托莫尼控股公司
英研化学公司
理创化学株式会社
日本板硝子公司
那家银行
富士科有限公司
礼品公司
雅萌有限公司
西格玛西控股有限公司
高斋堂控股有限公司
技研有限公司
日机装株式会社
田村株式会社
EXAWIZARDS公司
名古屋银行
爱知制钢所
新光工业有限公司
新东京工业株式会社
G-7 控股公司
赛源有限公司
内田洋工株式会社
大京西川株式会社
东惠电脑
京阪神大厦株式会社
日本工业株式会社
微路公司
中央汽车产品
德玛坎有限公司
西南CC公司
片仓工业公司
大机铝工业公司
爱达工程
理研维生素株式会社
日本光荣公司
日力机株式会社
GA科技有限公司
生命饮料有限公司
怎样医疗公司
市光工业
风格公司
奥泰克公司
松田产业
大阪有机化学工业公司
大阪制钢所
世界有限公司
关东电化工业公司
JSP公司
京洋株式会社
天马株式会社
萨姆科公司
伊尼有限公司
纳富科有限公司
SFP控股有限公司
日本能源株式会社
中国海洋涂料
博德罗伊公司
大黑电机株式会社
中部钢板公司
小森公司
日比谷工程
月岛控股株式会社
埃斯佩克公司
矢萩建设公司
交易有限公司
新日本航空技术有限公司
石原产业株式会社
全球安全专家公司
安华控股有

山田株式会社
大石环保。解决方案有限公司
文艺复兴公司
金子种子公司
群爱化学工业公司
右上公司
北关控股有限公司
格兰迪屋有限公司
联优公司
环境控股公司
联电电子有限公司
亚美兹有限公司
东亚DKK公司
东京基础设施能源投资公司
拉奎利亚制药公司
东京基地有限公司
三兴金属工业公司
烧肉堺控股株式会社
筑波银行
HIMACS有限公司
清水银行
英尼格莫公司
亚洲空气测量公司
泡菜控股有限公司
日本化学工业公司
交易控股
摩科瑞控股有限公司
巴黎三木控股有限公司
日本BS广播公司
OPTIMUS GP 有限公司
三和友嘉工业股份有限公司
AD-SOL日清株式会社
中央森林集团公司
新日本公司
焦点系统公司
交叉营销集团公司
京湾精工株式会社
吉克索有限公司
芬德克斯公司
FABRICA通讯有限公司
日本 MEIKO 网络
长岛丸株式会社
田边咨询集团有限公司
薪资公司
越知控股有限公司
浣熊控股公司
帝国通信工业公司
日本航空电子公司
寺崎电机株式会社
高岛公司
KNC实验室有限公司
开始出版公司
泰诺医疗有限公司
第一刀具工业株式会社
J丰泰有限公司
田喜公司
中外RO有限公司
纳米载体有限公司
诚信网络有限公司
尚高公司
四季野高科技有限公司
静木电机株式会社
卡纳生物科学公司
串胜田中控股株式会社
托尔有限公司
赛尔姆公司
拉莎公司
波平斯公司
聚泰控股公司
LTS公司
麦迪内特有限公司
互动公司
长堀株式会社
第一重研株式会社
JIG JP有限公司
日本数据科学联盟有限公司
日茂有限公司
JFLA控股有限公司
安乐亭有限公司
北陆燃气公司
高尔夫文摘在线公司
皮划艇公司
总研化学工业公司
日本房屋控股株式会社
好乐园控股公司
日亚制钢所
大研医疗有限公司
汉德曼有限公司
格诺西公司
J-STREAM 整合媒体公司
尤纳里公司
藤井产业株式会社
WDB可可有限公司
德利公司
中部日本放送有限公司
尤代郎化学工业公司
小野岛化学工业株式会社
泰康公司
阿米达控股有限公司
北药竹山控股株式会社
乌姆有限公司
日本滑雪场开发有限公司
陶氏有限公司
东邦系统科学公司
布鲁诺公司
神田控股有限公司
青木超级有限公司
泰赛克公司
今仙电机工业公司
前泽工业株式会社
鸟越有限公司（THE）
菲兹控股有限公司
德威公司
铃木株式会社
菱宇系统有限公司
新美控股有限公司
藤仓

山王株式会社
富田株式会社
新A化研有限公司
欣达有限公司
日本企业公司
转基因公司
格莱立方公司
中日本工业株式会社
网络嗡嗡声公司
永明金融控股有限公司
山下保健控股株式会社
斯瑞特公司
兴成证券
石墨设计公司
林克斯公司
川本株式会社
芬尼斯公司
奥村工程公司
PB系统公司
高北株式会社
商业一号控股有限公司
尼索普罗尼蒂有限公司
日本食品公司
卡丹美容有限公司
菲尔有限公司
俏丽生活有限公司
日本天线公司
卡拉达手特公司
东海软件有限公司
株式会社三木工业株式会社
土地商业有限公司
欧莱公司
中央综合发展有限公司
南开胶合板有限公司
中代公司
河电公司
丸八证券有限公司
生命食品有限公司
基因泰克公司
菊水化学工业公司
CSS控股有限公司
小野测器公司
富士精工有限公司
依康纳控股有限公司
环球信息公司
银座雷诺阿公司
里奇公司
富瓦大脑有限公司
华库尔公司
国际集团
菊川企业株式会社
伊内斯特公司
东武网络有限公司
北川精机公司
太阳机械工业公司
希望公司
纽拉布公司
富士变工株式会社
水手笔公司
阿斯莫公司
新福凯控股
京都工具有限公司
南日本银行
日本动物转诊医学中心有限公司
小苍兰超时空公司
土屋控股
DNA芯片研究公司
京商株式会社
韦尔比公司
玉川控股株式会社
三井株式会社
凯鸟工业株式会社
系统集成商公司
光工公司
系统位置
山庄电气公司
森谷株式会社
创造广场本社有限公司
SE控股和孵化
克林格尔制药公司
原创工程
秋叶控股有限公司
福留肉类包装机
塞雷斯波有限公司
利奥克兰有限公司
远州有限公司
日本甲州羽制钢所
横滨丸王株式会社
铁人株式会社
雷特公司
管理综合实验室
温测公司
株式会社HIRAKI
科科佩利公司
日和产业公司
回声贸易公司
通用牡蛎公司
伯格地球有限公司
一心控股有限公司
新格兰德酒店
教育实验室公司
梅尔迪亚发展建设有限公司
APPIRITS公司
丰成公司
基莫媒体公司
东京日产计算机系统有限公司
兰瑟斯公司
昭和系统工程公司
英特莱克斯有限公司
无处不在的人工智能公司
蓬莱有限公司
四季学株式会社
播磨B-STEM株式会社
松本公司
阪神柴油机有限公司
大研株式会社
维斯科科技股份有限公司
明治机械公司
创始人顾问控股有限公司
初穗精司公司
易美联公司
库欣公司
人民公司（玩具）
港区控股公司
苏拉拉网络有限公司
远

In [54]:
b_df = df[['logoid','country','is_primary','market_cap_USD','Traded_USD']].sort_values(['is_primary','logoid'],ascending = False)\
        .groupby(by = ['logoid','country','market_cap_USD','is_primary']).sum().sort_values(['market_cap_USD','logoid','Traded_USD'],ascending = False).reset_index().round(1)

b_df = b_df[b_df['market_cap_USD'] > 1]
b_df = b_df[b_df['Traded_USD'] > 0.01]
b_df = b_df[b_df['logoid'] != '']
b_df['is_primary'] = b_df['is_primary'].astype(str)
# b_df = b_df[b_df['country'] == 'United Kingdom']


b_df.groupby(by = ['logoid','country','market_cap_USD','is_primary']).sum().sort_values(['market_cap_USD','is_primary'],ascending = False).round(2).head(200)

# df[['logoid','country','is_primary','Traded_USD']].groupby(by = ['logoid','country','is_primary']).sum().sort_values(['Traded_USD'],ascending = False).round(2).head(200)

Traded_USD
logoid                                  country              market_cap_USD is_primary            
apple                                   United States        30512.1        False              1.0
                                                             30471.2        True              93.5
microsoft                               United States        26729.8        True             232.6
                                                             25706.7        False              1.6
saudi-arabian-oil                       Saudi Arabia         20799.8        True               0.5
alphabet                                United States        15852.1        False              0.4
                                                             15739.0        False             32.4
                                                             15727.9        True              26.1
amazon                                  United States        13703.7        False              7.1
                                                             13628.8        True              72.8
nvidia                                  United States        11731.0        True             270.2
                                                             11475.9        False              0.8
tesla                                   United States        9297.4         True             328.4
                                                             9203.6         False              4.0
meta-platforms                          United States        7998.3         True              64.7
                                                             7961.7         False              0.3
berkshire-hathaway                      United States        7552.1         False              8.9
                                                             7546.3         True              33.7
                                                             7528.3         False              0.6
taiwan-semiconductor                    Taiwan               4949.8         True               4.0
                                                                            False             10.7
visa                                    United States        4935.8         True              18.5
lvmh                                    France               4836.5         True               2.2
                                                                            False              1.0
unitedhealth                            United States        4655.2         True              44.1
                                                             4506.9         False              0.2
jpmorgan-chase                          United States        4490.4         True              20.7
                                                             4482.2         False              0.2
eli-lilly                               United States        4283.1         True              10.1
tencent                                 China                4281.6         True              12.5
                                                                            False              2.3
eli-lilly                               United States        4244.6         False              0.2
walmart                                 United States        4162.3         True               6.9
johnson-and-johnson                     United States        4133.5         True              10.1
exxon                                   United States        4098.8         False              0.1
                                                             4080.2         True              24.8
samsung                                 South Korea          3833.7         True               6.6
                                                                            False              0.5
mastercard                              United States        3771.3         True              10.7
broadcom                     

In [10]:
df.head(100)

,name,description,logoid,type,close,currency,change,Value.Traded,market_cap_basic,fundamental_currency_code,sector,industry,market,is_primary,exchange,country,currency_id,forex_rate,market_cap_USD,Traded_USD,turnover_rate,plate
0,0R2V,APPLE INC COM NPV,apple,stock,193.10001,USD,1.258526,6.436159e+07,3.051211e+12,USD,Electronic Technology,Telecommunications Equipment,uk,False,LSIN,United States,USD,1.0,30512.110162,0.643616,0.0000,欧洲
1,AAPL,APPLE INC,apple,stock,194.50000,USD,0.000000,5.835000e+02,3.051211e+12,USD,Electronic Technology,Telecommunications Equipment,peru,False,BVL,United States,USD,1.0,30512.110162,0.000006,0.0000,拉丁美洲
2,AAPL,APPLE INC,apple,stock,193.18000,USD,1.178448,9.659000e+02,3.051211e+12,USD,Electronic Technology,Telecommunications Equipment,chile,False,BCS,United States,USD,1.0,30512.110162,0.000010,0.0000,拉丁美洲
3,AAPLD,APPLE INC CEDEAR(REPR 1/10 SHR)USD,apple,dr,20.80000,USD,0.970874,5.401760e+04,3.051211e+12,USD,Electronic Technology,Telecommunications Equipment,argentina,False,BCBA,United States,USD,1.0,30512.110162,0.000540,0.0000,拉丁美洲
4,AAPL,Apple Inc.,apple,stock,193.73000,USD,-0.134028,9.348712e+09,3.047122e+12,USD,Electronic Technology,Telecommunications Equipment,america,True,NASDAQ,United States,USD,1.0,30471.215535,93.487118,0.0031,北美
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,AVGO,Broadcom Inc.,broadcom,stock,903.43000,USD,-0.760147,2.214074e+09,3.728324e+11,USD,Electronic Technology,Semiconductors,america,True,NASDAQ,United States,USD,1.0,3728.323558,22.140738,0.0059,北美
97,0TDD,NOVO NORDISK A/S ADR-EACH CNV INTO 1 CLASS'B'DKK1,novo-nordisk,dr,159.89999,USD,-0.713139,2.436876e+05,3.588718e+11,USD,Health Technology,Pharmaceuticals: Major,uk,False,LSE,Denmark,USD,1.0,3588.717593,0.002437,0.0000,欧洲
98,NVO,Novo Nordisk A/S,novo-nordisk,dr,160.25000,USD,-0.434918,1.398447e+08,3.588718e+11,USD,Health Technology,Pharmaceuticals: Major,america,False,NYSE,Denmark,USD,1.0,3588.717593,1.398447,0.0004,北美
99,NONOF,Novo-Nordisk A/S,novo-nordisk,stock,159.40000,USD,0.223837,1.853822e+05,3.545654e+11,USD,Health Technology,Pharmaceuticals: Major,america,False,OTC,Denmark,USD,1.0,3545.653694,0.001854,0.0000,北美


In [5]:
import plotly.express as px
# a_df = df#[df['plate'].isin(['Five Eyes'])]


fig = px.treemap(a_df, 
                 path=['plate','market','sector','industry'],  # 指定层次结构，每一个层次都应该是category型的变量
#                  path=['plate','','sector','industry',],
                 values='market_cap_USD', # 需要聚合的列名
                 color='change', 
                 custom_data=['change','name','market_cap_USD','close','market','close'],
                 range_color = [-8, 8], # 色彩范围最大最小值
#                  hover_data= {'涨跌幅':':.2',
#                              '总市值':':.1f'}, # 鼠标悬浮显示数据的格式
                 color_continuous_scale=["seagreen",'lightgrey', "indianred"],
                 height = 900,
                #  width = 1600,
                 color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
                )
fig.update_coloraxes(showscale=False)
fig.update_traces(textposition='middle center', 
                  textinfo='label', 
                  textfont = dict(color='white'),
                  texttemplate= "%{label}<br>%{customdata[0]:.2f}%<br>",
                  hovertemplate= "%{label}<br>%{customdata[5]:.2f} (%{customdata[0]:.2f}%)<br>%{customdata[4]}<br>总市值=%{customdata[2]:d}亿"                  
#                   hovertemplate= "%{customdata[1]}<br>%{label}<br>(%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿"
                 ) 
fig.show()

In [205]:
pd.set_option('display.max_rows',None)

def get_company_name(desc):
    return desc.replace('.',' ')\
                .replace('-',' ')\
                .replace('  ',' ')\
                .split(' ')[0].upper().replace(',','').replace("'",'')\
                .replace('BERKSHIRE','BERKSH')\
                .replace('TOTALENERGIES','TOTAL')\
                .replace('EXXILORLUXOTTICA','ESSILORLUXOTTICA')

df['company'] = df['description'].apply(get_company_name)


b_df = df[['company','country','is_primary','market_cap_USD','Traded_USD']].sort_values(['is_primary','company'],ascending = False)\
        .groupby(by = ['company','country','market_cap_USD']).sum().sort_values(['market_cap_USD','company','Traded_USD'],ascending = False).reset_index()

b_df = b_df[b_df['market_cap_USD'] > 1]
b_df = b_df[b_df['Traded_USD'] > 0.01]
b_df = b_df[b_df['country'] == 'United Kingdom']


b_df.groupby(by = ['company','country','market_cap_USD']).sum().sort_values(['market_cap_USD','Traded_USD'],ascending = False).round(2).head(2000)

is_primary  Traded_USD
company          country        market_cap_USD                        
ASTRAZENECA      United Kingdom 2258.596221              1        2.36
SHELL            United Kingdom 1968.758401              1        3.41
ROYAL            United Kingdom 1968.758401              0        1.09
LINDE            United Kingdom 1835.374859              1        0.68
                                1832.612061              0        0.09
HSBC             United Kingdom 1538.240032              1        0.45
                                1538.240032              0        1.37
UNILEVER         United Kingdom 1260.224324              1        0.96
                                1260.224324              0        0.52
RIO              United Kingdom 1138.305903              1        1.74
BP               United Kingdom 1016.265501              1        3.07
DIAGEO           United Kingdom 959.787308               1        1.07
BRITISH          United Kingdom 725.710159               1        0.88
GSK              United Kingdom 702.769226               1        0.49
RELX             United Kingdom 617.820227               1        0.75
LONDON           United Kingdom 554.101777               1        0.26
RECKITT          United Kingdom 536.502338               1        0.66
NATIONAL         United Kingdom 480.376436               1        0.64
COMPASS          United Kingdom 479.113677               1        0.20
ANGLO            United Kingdom 437.348287               1        0.60
PRUDENTIAL       United Kingdom 394.179476               1        0.23
HALEON           United Kingdom 372.854500               1        0.23
LLOYDS           United Kingdom 372.250301               1        1.40
BAE              United Kingdom 366.433744               1        0.12
BARCLAYS         United Kingdom 309.680262               1        1.20
ASHTEAD          United Kingdom 303.177591               1        0.13
COCA             United Kingdom 301.553118               1        0.08
NATWEST          United Kingdom 300.521301               1        0.45
FERGUSON         United Kingdom 296.736505               1        0.28
                                294.401388               0        0.08
LYONDELLBASELL   United Kingdom 292.950055               1        0.15
SSE              United Kingdom 251.735168               1        0.08
VODAFONE         United Kingdom 248.948900               0        0.23
                                248.948900               1        0.34
3I               United Kingdom 248.688377               1        0.18
WILLIS           United Kingdom 245.218238               1        0.13
STANDARD         United Kingdom 238.540659               1        0.28
TESCO            United Kingdom 238.282953               1        0.49
RENTOKIL         United Kingdom 203.838187               1        0.13
IMPERIAL         United Kingdom 201.679542               1        0.27
CNH              United Kingdom 194.726171               0        0.20
ANTOFAGASTA      United Kingdom 194.718137               1        0.07
CARNIVAL         United Kingdom 193.874780               1        0.20
CNH              United Kingdom 192.113745               1        0.28
ASSOCIATED       United Kingdom 184.080014               1        0.06
LEGAL            United Kingdom 178.933701               1        0.24
BT               United Kingdom 171.875607               1        0.10
ROLLS            United Kingdom 159.707771               1        0.13
AMCOR            United Kingdom 149.719397               1        0.06
                                149.498680               0        0.21
AVIVA            United Kingdom 138.465180               1        0.07
SMITH            United Kingdom 137.922986               1        0.15
BUNZL            United Kingdom 132.168751               1        0.12
INFORMA          United Kingdom 125.206509               1        0.21
INTERCONTINENTAL United Kingdom 119.858383               1        0.13
W